## Notebook to create better prompts

The idea of this notebook is to use `langchain` to improve promtps for GenerativeAI models to then create better results for text-generation or image-generation. 
The idea is inspired by the [PromptPerfect](https://promptperfect.jina.ai/) tool from jina.

We are going to use the Self-Critique Chain with Constitutional AI to create better prompts for text-generation and image-generation

In [ ]:
!pip install langchain huggingface_hub --quiet

## Few-shot prompting

> Can you help me worsen the prompt below? The prompt is instruction and fed into a GPT model to answer the instruction. I want to generate bad prompts to use reinforcement learning later. Below are examples of what I want you to do: 

```
You are asked to optimize user instruction to feed into a GPT model to answer them. The user input instruction need to be improved in a way that the GPT model generates higher-quality results, which are more helpful, harmless. 
Here are the requirements:
1. Try not to repeat the verb for each instruction to maximize diversity.
2. The language used for the instruction also should be diverse. For example, you should combine questions with imperative instrucitons.
3. The instructions should be in English.
4. The instructions should be specific and clear. For example, do not ask the assistant to create general prompts.
5. Avoid using complex language or jargon in the instruction prompt. This can confuse the GPT model and lead to less helpful responses. Use simple and clear language to ensure that the model understands what you are asking for.
6. Providing examples of the type of response you are looking for can help the GPT model generate more relevant and helpful responses. For example, if you are asking for tips on staying healthy, provide a few examples of what a good tip might look like.
7. Consider the length and complexity of the prompt: It's important to balance the length and complexity of the instruction prompt with the desired quality of the response. A shorter and simpler prompt may lead to more concise and focused responses, while a longer and more complex prompt may allow for more nuanced and detailed responses.

Here are some examples of user prompts and improved prompts:
user: Give three tips for staying healthy.
optimized: Please provide three practical and actionable tips for maintaining good health. Your response should include specific actions that can be taken to promote physical, mental, and emotional well-being. Consider including advice related to exercise, nutrition, stress management, and other lifestyle factors that can impact overall health.
user: create a story with only words less than 3 letters.
optimized: Please create a short story using only words that are less than three letters long. Your story should be creative and engaging, using simple language to convey complex ideas. You may use punctuation marks as needed to enhance the flow of your story. Your response should demonstrate an ability to communicate effectively with limited vocabulary, while still conveying a coherent narrative. Please ensure that your story has a clear beginning, middle, and end, and includes characters, conflict or tension, and resolution.
--- 
user: summarize the news article below
```


In [8]:
from langchain import PromptTemplate, FewShotPromptTemplate


prefix_prompt="""You are asked to optimize user instruction to feed into a GPT model to answer them. The user input instruction need to be improved in a way that the GPT model generates higher-quality results, which are more helpful, harmless. 
Here are the requirements:
1. Try not to repeat the verb for each instruction to maximize diversity.
2. The language used for the instruction also should be diverse. For example, you should combine questions with imperative instrucitons.
3. The instructions should be in English.
4. The instructions should be specific and clear. For example, do not ask the assistant to create general prompts.
5. Avoid using complex language or jargon in the instruction prompt. This can confuse the GPT model and lead to less helpful responses. Use simple and clear language to ensure that the model understands what you are asking for.
6. Providing examples of the type of response you are looking for can help the GPT model generate more relevant and helpful responses. For example, if you are asking for tips on staying healthy, provide a few examples of what a good tip might look like.
7. Consider the length and complexity of the prompt: It's important to balance the length and complexity of the instruction prompt with the desired quality of the response. A shorter and simpler prompt may lead to more concise and focused responses, while a longer and more complex prompt may allow for more nuanced and detailed responses.

Here are some examples of user prompts and improved prompts:"""
suffux="user: {input}\optimized:"

# First, create the list of few shot examples.
examples = [
    {"user": "Give three tips for staying healthy.",
     "optimized": "Please provide three practical and actionable tips for maintaining good health. Your response should include specific actions that can be taken to promote physical, mental, and emotional well-being. Consider including advice related to exercise, nutrition, stress management, and other lifestyle factors that can impact overall health."},
    {"user": "create a story with only words less than 3 letters.", "optimized": "lease create a short story using only words that are less than three letters long. Your story should be creative and engaging, using simple language to convey complex ideas. You may use punctuation marks as needed to enhance the flow of your story. Your response should demonstrate an ability to communicate effectively with limited vocabulary, while still conveying a coherent narrative. Please ensure that your story has a clear beginning, middle, and end, and includes characters, conflict or tension, and resolution."},
]

# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """user: {user}
optimized: {optimized}"""
example_prompt = PromptTemplate(
    input_variables=["user", "optimized"],
    template=example_formatter_template,
)

# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix=prefix_prompt,
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="user: {input}\noptimized:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

# We can now generate a prompt using the `format` method.
print(few_shot_prompt.format(input="Describe the function of a computer motherboard"))
# -> Give the antonym of every input

You are asked to optimize user instruction to feed into a GPT model to answer them. The user input instruction need to be improved in a way that the GPT model generates higher-quality results, which are more helpful, harmless. 
Here are the requirements:
1. Try not to repeat the verb for each instruction to maximize diversity.
2. The language used for the instruction also should be diverse. For example, you should combine questions with imperative instrucitons.
3. The instructions should be in English.
4. The instructions should be specific and clear. For example, do not ask the assistant to create general prompts.
5. Avoid using complex language or jargon in the instruction prompt. This can confuse the GPT model and lead to less helpful responses. Use simple and clear language to ensure that the model understands what you are asking for.
6. Providing examples of the type of response you are looking for can help the GPT model generate more relevant and helpful responses. For example, i

In [13]:
import os 
os.environ = {**os.environ, **{env.split("=")[0]: env.split("=")[1].replace("\n","") for env in open(".env", "r").readlines()}}

In [45]:
from langchain.llms import OpenAI
llm = OpenAI()

res = llm(few_shot_prompt.format(input="Write about how to house train a dog."))

In [46]:
print(res)

 Please provide detailed instructions on how to house train a dog. Your response should include clear steps that can be taken to help a dog learn the appropriate behaviors for living in a home environment. Consider including advice on setting boundaries, consistent reinforcement, and other strategies for teaching a dog how to use the restroom outside. In addition, provide tips on how to deal with any accidents that may occur during the training process.


## Result on [tloen/alpaca-lora](https://huggingface.co/spaces/tloen/alpaca-lora)

input: `Describe the function of a computer motherboard`  

output: `A computer motherboard is the main circuit board of a computer. It is responsible for connecting all of the components of a computer, such as the processor, memory, graphics card, and other peripherals. It also provides power and data connections between the components.`

---

with improved prompt: `Please describe the function of a computer motherboard. Your response should include a detailed description of the motherboard's purpose and function, as well as a description of the components that make up the motherboard. Please ensure that your response is clear and concise, and that it includes all relevant information.` 

output: `A computer motherboard is the main circuit board of a computer system. It is responsible for connecting all of the components of a computer, such as the CPU, RAM, and other peripherals. The motherboard acts as the central nervous system of the computer, allowing all of the components to communicate with each other and with the CPU. The motherboard is also responsible for distributing power to the various components of the computer.`

## Use ChatGPT to improve prompts

In [40]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template="""You are asked to optimize user instruction to feed into a GPT model to answer them. The user input instruction need to be improved in a way that the GPT model generates higher-quality results, which are more helpful, harmless. 
Here are the requirements:
1. Try not to repeat the verb for each instruction to maximize diversity.
2. The language used for the instruction also should be diverse. For example, you should combine questions with imperative instrucitons.
3. The instructions should be in English.
4. The instructions should be specific and clear. For example, do not ask the assistant to create general prompts.
5. Avoid using complex language or jargon in the instruction prompt. This can confuse the GPT model and lead to less helpful responses. Use simple and clear language to ensure that the model understands what you are asking for.
6. Providing examples of the type of response you are looking for can help the GPT model generate more relevant and helpful responses. For example, if you are asking for tips on staying healthy, provide a few examples of what a good tip might look like.
7. Consider the length and complexity of the prompt: It's important to balance the length and complexity of the instruction prompt with the desired quality of the response. A shorter and simpler prompt may lead to more concise and focused responses, while a longer and more complex prompt may allow for more nuanced and detailed responses.

Here are some examples of user prompts and improved prompts:
user: Give three tips for staying healthy.
optimized: Please provide three practical and actionable tips for maintaining good health. Your response should include specific actions that can be taken to promote physical, mental, and emotional well-being. Consider including advice related to exercise, nutrition, stress management, and other lifestyle factors that can impact overall health.
user: create a story with only words less than 3 letters.
optimized: lease create a short story using only words that are less than three letters long. Your story should be creative and engaging, using simple language to convey complex ideas. You may use punctuation marks as needed to enhance the flow of your story. Your response should demonstrate an ability to communicate effectively with limited vocabulary, while still conveying a coherent narrative. Please ensure that your story has a clear beginning, middle, and end, and includes characters, conflict or tension, and resolution.
"""

# We can now generate a prompt using the `format` method.
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template="user: {input}\noptimized:"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
messages = chat_prompt.format_prompt(input="Describe the function of a computer motherboard").to_messages()

In [44]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI()

res = chat(messages)
res.content

'Please provide a clear and concise explanation of the purpose and function of a computer motherboard. Your response should describe how the motherboard serves as a central hub for connecting and coordinating different computer components, including the CPU, memory, input/output devices, and storage devices. Consider including examples of how the motherboard allocates resources, manages data transfer, and facilitates communication between different parts of the computer system. Please ensure that your explanation is comprehensive yet accessible, avoiding jargon or technical language that may be difficult for non-experts to understand.'

with improved prompt: `Please provide a concise and precise explanation of what a computer motherboard is and what its main functions are. Your response should be easy to understand and avoid using technical jargon. Consider including information about how a motherboard connects and controls the components of a computer, the types of components that can be installed on a motherboard, and the role of the motherboard in managing data and communication within a computer system.` 

output: `A computer motherboard is the main circuit board of a computer system. It is responsible for connecting and controlling the various components of a computer, such as the processor, memory, storage, and peripherals. The motherboard also acts as a central hub for data and communication within a computer system, allowing information to be transferred between the various components.`

## Test on alpaca dataset with chatGPT responses

In [6]:
from datasets import load_dataset
from random import randint

# load the dataset
dataset = load_dataset("yahma/alpaca-cleaned",split="train")

# pick a random example
sample = dataset[randint(0, len(dataset))]
print(sample)

Found cached dataset json (/Users/philipp/.cache/huggingface/datasets/yahma___json/yahma--alpaca-cleaned-c372ba229cec7800/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


{'instruction': 'Outline the benefits of using a web-based application.', 'input': '', 'output': "Using a web-based application can offer several benefits. It allows users to access the application anywhere, as long as they have an internet connection. It is also more cost-effective than traditional applications, as there is no need to purchase or maintain hardware or software. Additionally, web-based applications are typically much easier to update and maintain, as there is no need to install a patch or update on each user's device. Finally, web-based applications provide greater control for the user, as they can decide which features and functions are most important to them."}


use chatGPT improve prompt chain to improve prompt for alpaca dataset and then generate the response with chatGPT

In [14]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chat_models import ChatOpenAI


system_template="""You are asked to optimize user instruction to feed into a GPT model to answer them. The user input instruction need to be improved in a way that the GPT model generates higher-quality results, which are more helpful, harmless. 
Here are the requirements:
1. Try not to repeat the verb for each instruction to maximize diversity.
2. The language used for the instruction also should be diverse. For example, you should combine questions with imperative instrucitons.
3. The instructions should be in English.
4. The instructions should be specific and clear. For example, do not ask the assistant to create general prompts.
5. Avoid using complex language or jargon in the instruction prompt. This can confuse the GPT model and lead to less helpful responses. Use simple and clear language to ensure that the model understands what you are asking for.
6. Providing examples of the type of response you are looking for can help the GPT model generate more relevant and helpful responses. For example, if you are asking for tips on staying healthy, provide a few examples of what a good tip might look like.
7. Consider the length and complexity of the prompt: It's important to balance the length and complexity of the instruction prompt with the desired quality of the response. A shorter and simpler prompt may lead to more concise and focused responses, while a longer and more complex prompt may allow for more nuanced and detailed responses.

Here are some examples of user prompts and improved prompts:
user: Give three tips for staying healthy.
optimized: Please provide three practical and actionable tips for maintaining good health. Your response should include specific actions that can be taken to promote physical, mental, and emotional well-being. Consider including advice related to exercise, nutrition, stress management, and other lifestyle factors that can impact overall health.
user: create a story with only words less than 3 letters.
optimized: lease create a short story using only words that are less than three letters long. Your story should be creative and engaging, using simple language to convey complex ideas. You may use punctuation marks as needed to enhance the flow of your story. Your response should demonstrate an ability to communicate effectively with limited vocabulary, while still conveying a coherent narrative. Please ensure that your story has a clear beginning, middle, and end, and includes characters, conflict or tension, and resolution.
"""

# We can now generate a prompt using the `format` method.
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template="user: {input}\noptimized:"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


# creates chat model
chat = ChatOpenAI()

# creates chat prompt messages
def improve_prompt(prompt, llm=chat): 
  messages=chat_prompt.format_prompt(input=prompt).to_messages()
  return llm(messages).content


generate improve prompt

In [15]:
# generate improved prompt
prompt = improve_prompt(sample["instruction"])

In [22]:
prompt

'Please describe the advantages of utilizing a web-based application. Your response should include specific benefits related to accessibility, ease of use, maintenance, and cost-effectiveness. Consider discussing how web-based applications do not require installation, can be accessed from anywhere with an internet connection, and do not require updates or upgrades to be performed by the user. Additionally, highlight how web-based applications can be more affordable than traditional software options and how they often provide better scalability and customization options. Please provide examples to illustrate your points.'

generate new response

In [24]:
from langchain.schema import (
    HumanMessage,
)

output = chat([HumanMessage(content=sample["instruction"])])

In [27]:
print(output.content)

1. Accessibility: Web-based applications are accessible from any location with an internet connection.

2. Platform Independence: They run on various platforms, including Windows, Mac, and Linux, providing compatibility with multiple operating systems.

3. Cost-effectiveness: No need to purchase and maintain separate resources such as servers and databases.

4. Reduced installation and maintenance costs: Since the client-side has a minimal application, it eliminates the need for software installations and updates.

5. Data centralized management: Data for web applications is stored on a central server that ensures uniform data access and makes data backup and recovery more straightforward.

6. Scalability: With web-based applications, businesses can scale up or down depending on their needs without affecting their operations.

7. Real-time data processing: Web applications allow for real-time processing of data on servers, enabling businesses to select and access complete data suites a

In [17]:
from langchain.schema import (
    HumanMessage,
)

output = chat([HumanMessage(content=prompt)])

In [23]:
print(output.content)

Web-based applications have numerous advantages over traditional software options. Some of the specific benefits related to accessibility, ease of use, maintenance, and cost-effectiveness are:

1. Accessibility: Web-based applications can be accessed from anywhere with an internet connection, which makes it easier for businesses that operate from multiple locations or have employees working remotely. This eliminates the need for installation on individual computers and ensures that all users access the same application regardless of their location.

2. Ease of use: Web-based applications are typically easier to use and navigate than desktop software. With a user-friendly interface, web-based applications offer enhanced interactivity, faster response time, and simplified workflow. Users can easily share files, collaborate, and communicate in real-time.

3. Low maintenance: Web-based applications do not require installation or upgrades, saving users the hassle of performing routine updat

## Constitution AI experiment

TBD

# Image Prompt improvements

There is already a models:
-  https://huggingface.co/microsoft/Promptist
-  https://huggingface.co/RamAnanth1/distilgpt2-sd-prompts

In [52]:
from langchain import PromptTemplate, FewShotPromptTemplate


prefix_prompt="""You are asked to optimize user inputs, which are then used to generate images. The user input need to be improved in a way that the image model generates higher-quality results. 
Here are some examples of user prompts and improved prompts:"""
suffux="user: {input}\optimized:"

# First, create the list of few shot examples.
examples = [
    {"user": "A rabbit is wearing a space suit.", "optimized": "A rabbit is wearing a space suit, digital Art, Gregrutkowski, Trending cinematographic artstation."},
    {"user": "Several railroad tracks with one train passing by", "optimized": "several railroad tracks with one train passing by, hyperdetailed, artstation, cgsociety, 8 k"},
    {"user": "The roof is wet from the rain", "optimized": "the roof is wet from the rain, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration,"},
    {"user": "Cats dancing in a space club", "optimized": "Cats dancing in a space club, digital painting, artstation, concept art, soft light, hdri, smooth, sharp focus, illustration, fantasy,"},
    # {"user": "temple in ruines, forest", "optimized": "a surreal digital painting artwork of a mysterious temple in ruins, deep in the dark forest with foggy atmosphere and colorful autumn leaves falling from trees, rendered in photoshop by artist WLOP, John Jiao, Kode Logic, Archan Nair, Ilya Kuvshinov, vibrant colors palette, subtle details, perfect composition, dynamic lighting effects –ar 4:3 –upbeta"},
]

# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """user: {user}
optimized: {optimized}"""
example_prompt = PromptTemplate(
    input_variables=["user", "optimized"],
    template=example_formatter_template,
)

# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix=prefix_prompt,
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="user: {input}\noptimized:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

# We can now generate a prompt using the `format` method.
print(few_shot_prompt.format(input="a rocket launching into space"))
# -> Give the antonym of every input

You are asked to optimize user inputs, which are then used to generate images. The user input need to be improved in a way that the image model generates higher-quality results. 
Here are some examples of user prompts and improved prompts:
user: A rabbit is wearing a space suit.
optimized: A rabbit is wearing a space suit, digital Art, Gregrutkowski, Trending cinematographic artstation.
user: Several railroad tracks with one train passing by
optimized: several railroad tracks with one train passing by, hyperdetailed, artstation, cgsociety, 8 k
user: The roof is wet from the rain
optimized: the roof is wet from the rain, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration,
user: Cats dancing in a space club
optimized: Cats dancing in a space club, digital painting, artstation, concept art, soft light, hdri, smooth, sharp focus, illustration, fantasy,
user: a rocket launching into space
optimized:


In [53]:
from langchain.llms import OpenAI
llm = OpenAI()

res = llm(few_shot_prompt.format(input="a rocket launching into space"))